In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Process the dataset

In [ ]:
train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
#train2 = train.drop(['row_id'], axis = 1).sort_values(by=['investment_id','time_id'])
#train2.set_index(["investment_id", "time_id"], inplace = True)
#investment_ID = train2.index

# drop outlier observation

thanks for JUNJI TAKESHIMA's work, and please find the details here:
https://www.kaggle.com/junjitakeshima/ubiquant-simple-lgbm-removing-outliers-en-jp

In [ ]:
features = ['f_{}'.format(i) for i in range(300)]

In [ ]:
outlier_list = []
outlier_col = []

for col in features :
    
    temp_df = train[(train[col] > train[col].mean() + train[col].std() * 50) |
                       (train[col] < train[col].mean() - train[col].std() * 50) ]

    if len(temp_df) >0 : 
        outliers = temp_df.index.to_list()
        outlier_list.extend(outliers)
        outlier_col.append(col)
        print(col, len(temp_df))

In [ ]:
train.drop(train.index[outlier_list], inplace = True)

In [ ]:
train2 = train.drop(['row_id'], axis = 1).sort_values(by=['investment_id','time_id'])
train2.set_index(["investment_id", "time_id"], inplace = True)
investment_ID = train2.index

## Find all investment_id with observation smaller than 30 to prepare for later execution

In [ ]:
invest_id_count = train['investment_id'].value_counts()
low_observation_id = list(invest_id_count[invest_id_count < 30].index)
low_observation_id

In [ ]:
investment_IDs = sorted(list(train.investment_id.unique()))
print(len(investment_IDs))

In [ ]:
features = ['f_{}'.format(i) for i in range(300)]

## Define select_high_var_features, select_high_corr_features, select_high_mi_features functions
## to select those features with high corr

In [ ]:
# apply mutual_info to analyse feature
from sklearn.feature_selection import mutual_info_regression
def make_mi_score_real(X, y):
    mi_scores = mutual_info_regression(X, y)
    mi_scores = pd.Series(mi_scores, name = 'MI scores', index = X.columns)
    mi_scores = mi_scores.sort_values(ascending = False)
    return mi_scores
# Now we define a function to selection the features with a high variance, 
# a high correlation with the target, and a high mutual information score for each investment ID. 

    
def select_high_var_features(investment_ID, n_qcut):
    #drop low variance features
    var = train2[features].loc[investment_ID].var().sort_values() 
    
    threshold = pd.qcut(var, n_qcut, retbins = True)
    
    low_var_features = [feature for feature in features if var[feature] < threshold[1][1]]
    high_var_features = [feature for feature in features if var[feature] >= threshold[1][1]]
    
    return high_var_features


def select_high_corr_features(investment_ID, high_var_features = 0, corr_cut = 0, corr_rank = 0):
    """ corr_cut: the divisor of max_correlation
    """
    #correlation between the target and each feature: drop low-cor features
    corr_d = {}
    train4 = train2[high_var_features]
    for i in high_var_features[1:]:                                         
        corr_d['{}'.format(i)] = abs(train4.loc[investment_ID].loc[:,['target',i]].corr().iloc[0,1])
    corr_series = pd.Series(corr_d)
    
    threshold = corr_series.max() / corr_cut
    high_corr_list = [feature for feature in corr_d.keys() if corr_d[feature] > threshold]
    
    return high_corr_list


def select_high_mi_features(investment_ID, mi_cut = 0, mi_rank = 0):
    
    mi_scores_real = make_mi_score_real(train2.drop(['target'], axis =1).loc[investment_ID], train2.loc[investment_ID].target)
    mi_scores_real.round(decimals = 4)
    mi_scores_real = mi_scores_real[:mi_rank]
    mi_scores_df = pd.DataFrame({'mi_scores':mi_scores_real})
    #plt.figure(figsize = (20,20))
    #sns.barplot(y = mi_scores_df.index, x = mi_scores_df['mi_scores'])
    
    return list(mi_scores_real.index)


def select_features(investment_ID, n_qcut, corr_cut = 0, mi_rank = 0, corr_rank = 0, mi_cut = 0):
    
    high_var_features = select_high_var_features(investment_ID, n_qcut)
    
    high_var_features.insert(0, 'target')
    
    high_corr_features = select_high_corr_features(investment_ID, high_var_features = high_var_features,
                                                   corr_cut = corr_cut, corr_rank = corr_rank)
    
    b = high_corr_features.copy()
    
    high_mi_features = select_high_mi_features(investment_ID, mi_cut = mi_cut, mi_rank = mi_rank)
    
    c = high_mi_features.copy()

    conjunction_list = list(set(b) & set(c))
    
    #print(conjunction_list)
    #print(f"Number of high corr features: {len(b)};\nNumber of high mi score features: {len(c)};\nNumber of common features: {len(conjunction_list)}")
    
    return sorted(conjunction_list)
    


## in this case, make all other features be zero so that the regressor will omit it when recognize this specific investment_id

In [ ]:
new_train2 = train2.copy()
for investment_ID in investment_IDs:
    if investment_ID in low_observation_id:
        continue
    else:
        high_relevance_list = select_features(investment_ID, n_qcut = 6, corr_cut = 5, mi_rank = 70)
        new_train2.loc[investment_ID].loc[:, high_relevance_list] = 0
        print("The length of the list {} is {}.".format(investment_ID,  len(high_relevance_list)))

In [ ]:
print("all set")

In [ ]:
new_train2.to_parquet('output.parquet')